In [1]:
import numpy as np
import pandas as pd

In [2]:
hea_22 = pd.read_stata('ch22o_EN_1.0p.dta')
rel_22 = pd.read_stata('cr22o_EN_1.0p.dta')
soc_22 = pd.read_stata('cs22o_EN_1.1p.dta')
fam_22 = pd.read_stata('cf22o_EN_1.0p.dta')
wor_22 = pd.read_stata('cw22o_EN_1.0p.dta')
per_22 = pd.read_stata('cp22n_EN_1.0p.dta')
val_22 = pd.read_stata('cv22n_EN_1.0p.dta')
ass_22 = pd.read_stata('ca22h_EN_1.0p.dta')
inc_22 = pd.read_stata('ci22o_EN_1.0p.dta')
hou_22 = pd.read_stata('cd22o_EN_1.0p.dta')

In [ ]:
all_22 = [ass_22, hou_22, fam_22, hea_22, inc_22, per_22, soc_22, val_22, wor_22]

merged_22 = all_22[0]

for all in all_22[1:]:
    merged_22 = pd.merge(merged_22, all, on='nomem_encr', how='inner')

In [ ]:
merged_22.head()

In [ ]:
m22_80nan = merged_22.columns[merged_22.isna().mean() > 0.8].tolist()
clean_22 = merged_22.drop(columns=m22_80nan, axis=1)
clean_22.head()

In [ ]:
#column combinators (replaces given columns with one containing their avg values)
def average_columns(df, new_column, *columns):
    df[new_column] = df[columns].mean(axis=1)
    df.drop(columns=list(columns), axis=1, inplace=True)

def sum_columns(df, new_column, *columns):
    df[new_column] = df[columns].sum(axis=1)

concepts in health:
1. subjective health (ch22o004 to sub_hea)
2. relative health (ch22o005 to rel_hea)
3. life expectancy
4. mental health (all neg as men_hea_neg and and all pos as men_hea_pos)
5. height, weight
6. chronical problems (ch22o018 to disability)
7. health problems
8. impediment in labour
9. sex drugs alcohol ()

In [ ]:
hea_toavg = [('ch22o011','ch22o14','ch22o12', 'men_hea_neg'),('ch22o013', 'ch22o015', 'men_hea_pos'), ]
hea_tosum = [()]
hea_todrop = []

for set_tuple in hea_tocombine:
    average_columns(clean_22, *set_tuple)

print(clean_22)

In [ ]:
hea_con = {'ch22o004': 'sub_hea', 'ch22o005': 'rel_hea', 'ch22o018': 'disability', 'ch22o020':'mob_issues', 'ch22o126':'smoker', 'ch22o133':'alcohol', 'drugs' }


In [ ]:
#no of cars, motorcycles, boats, (static) caravans is ca22h008
#value is ca22h023 ??????????????what now

Concepts in religion and ethnicity:

1. Religious upbringing (cr23p135 to rel_upb)
2. Religious affiliation (cr23p144 to rel_aff)
3. Religiosity (cr23p162 to relig)
4. Religious orthodoxy
5. Nationality 
6. Origin (cr23p079 – cr23p087 to lang_ori, cr23p079 indicating Dutch or not)
7. Ethnic identification (cr23p164 to eth_id)
8. Language proficiency and use

In [22]:
codebook_relig = {'certainly religious':1, 'somewhat religious':2, 'barely religious':3, 'certainly not religious':4}
rel_22['cr22o162'] = rel_22['cr22o162'].replace(codebook_relig)
rel_con = {'cr22o135':'rel_upb', 'cr22o144':'rel_aff', 'cr22o162':'relig', 'cr22o079':'lang_ori', 'cr22o164':'eth_id'}
selected_columns = list(rel_con.keys())
rel_22_con = rel_22.loc[:, selected_columns].rename(columns=rel_con)
rel_22_con.head()



,rel_upb,rel_aff,relig,lang_ori,eth_id
0,NaN,NaN,4,Yes,Very strongly
1,Dutch Reformed Church (from 2004 the Protestan...,Dutch Reformed Churches,1,Yes,Fairly strongly
2,"Reformed Churches in the Netherlands, synodal ...",Protestant Church in the Netherlands (this inc...,I dont know,Yes,Not so strongly
3,Roman Catholic,NaN,4,No,Not strongly at all
4,NaN,NaN,4,Yes,Fairly strongly


Concepts in social intergation and leisure:

1. Loneliness
2. Social contacts 
3. Satisfaction leisure (cs22o001 to lei_sat)
4. Social engagement
5. Volunteer aid
6. Time expenditure voluntary work
7. Cultural participation (sum cs22o494 - cs22o496 to cul_par)
8. Holidays (cs22o103 to ab_hol)
9. Sport (cs22o105 to spo_h)
10. Media usage

In [5]:
codebook_cul = {'0 times': 0, '1 time': 1, '2 to 3 times': 2, '4 to 11 times': 3, '12 times or more': 4}
codebook_lei = {'not at all satisfied':0, 'completely satisfied':10}
codebook_hol = {'not once':0, 'one time':1, 'two times':2, 'three times':3, 'four times':4, 'five times or more':'≥5'}
cul_sum = ['cs22o494', 'cs22o495', 'cs22o517', 'cs22o093', 'cs22o094', 'cs22o568', 'cs22o516', 'cs22o496']
soc_22[cul_sum] = soc_22[cul_sum].replace(codebook_cul)
soc_22['cs22o001'] = soc_22['cs22o001'].replace(codebook_lei)
soc_22['cs22o103'] = soc_22['cs22o103'].replace(codebook_hol)
soc_22['cul'] = soc_22[cul_sum].sum(axis=1)
soc_con = {'cs22o001':'lei_sat', 'cul':'cul_par', 'cs22o103':'ab_hol', 'cs22o105':'spo_h'}
selected_columns = list(soc_con.keys())
soc_22_con = soc_22.loc[:, selected_columns].rename(columns=soc_con)
    
print(soc_22_con)

     lei_sat  cul_par ab_hol  spo_h
0        8.0      1.0      0    3.0
1        7.0      0.0      1    NaN
2         10      2.0      2    NaN
3        4.0      0.0      1    NaN
4        9.0      6.0      0    NaN
...      ...      ...    ...    ...
5879     8.0      4.0      1    3.0
5880     7.0      5.0      2    6.0
5881     2.0      0.0      1    4.0
5882     7.0      3.0      3   10.0
5883     8.0      8.0      3    5.0

[5884 rows x 4 columns]
